In [102]:
from openpyxl import load_workbook, Workbook
import os

# === Ruta de tu archivo ===
ruta_excel = r"C:\Users\diego\OneDrive\9NO CICLO\ANALITICA DE DATOS\PARCIAL\PARTE 2\tarifarios_consolidado.xlsx"

# === Salida (nuevo archivo para no tocar el original) ===
carpeta = os.path.dirname(ruta_excel)
salida = os.path.join(carpeta, "tarifarios_consolidado.xlsx")

# 1) Abrir el libro origen y obtener hojas en orden
wb_in = load_workbook(ruta_excel, data_only=True, read_only=False)
nombres = wb_in.sheetnames  # orden original

# 2) Crear libro de salida con una sola hoja
wb_out = Workbook()
ws_out = wb_out.active
ws_out.title = "PEGADO"

dest_row = 1  # fila destino donde se irá pegando

# 3) Copiar, en orden, el contenido de cada hoja tal cual (solo valores)
for nombre in nombres:
    ws_in = wb_in[nombre]
    max_r = ws_in.max_row or 0
    max_c = ws_in.max_column or 0
    if max_r == 0 or max_c == 0:
        continue  # hoja vacía

    for r in ws_in.iter_rows(min_row=1, max_row=max_r, min_col=1, max_col=max_c, values_only=True):
        for c_idx, val in enumerate(r, start=1):
            ws_out.cell(row=dest_row, column=c_idx, value=val)
        dest_row += 1  # siguiente fila al terminar una fila de la hoja actual

# 4) Guardar
wb_out.save(salida)
print(f"[OK] Pegado completado en: {salida}")


[OK] Pegado completado en: C:\Users\diego\OneDrive\9NO CICLO\ANALITICA DE DATOS\PARCIAL\PARTE 2\tarifarios_consolidado.xlsx


In [103]:
from openpyxl import load_workbook, Workbook
import os

# === Input (tu consolidado que ahora sobreescribes) ===
ruta_input = r"C:\Users\diego\OneDrive\9NO CICLO\ANALITICA DE DATOS\PARCIAL\PARTE 2\tarifarios_consolidado.xlsx"
nombre_hoja_origen = "PEGADO"   # hoja donde pegaste todo

# === Output (archivo nuevo con las hojas separadas) ===
carpeta = os.path.dirname(ruta_input)
ruta_output = os.path.join(carpeta, "tarifarios_consolidado.xlsx")

# Marcadores a separar
marcadores = [f"3.{i}" for i in range(1, 19)]  # 3.1 ... 3.18

# 1) Abrir libro y hoja origen
wb_in = load_workbook(ruta_input, data_only=True, read_only=False)
if nombre_hoja_origen not in wb_in.sheetnames:
    raise SystemExit(f"[ERROR] No encuentro la hoja '{nombre_hoja_origen}' en el archivo de entrada.")

ws_src = wb_in[nombre_hoja_origen]
max_r = ws_src.max_row or 0
max_c = ws_src.max_column or 0
if max_r == 0 or max_c == 0:
    raise SystemExit("[ERROR] La hoja de origen está vacía.")

def empieza_con(val, target):
    if val is None:
        return False
    return str(val).strip().startswith(target)

# 2) Encontrar filas de inicio para cada marcador
indices = {}  # {"3.n": fila_inicio}
for i in range(1, max_r + 1):
    a_val = ws_src.cell(row=i, column=1).value  # Columna A
    for m in marcadores:
        if m not in indices and empieza_con(a_val, m):
            indices[m] = i

# Ordenar marcadores realmente encontrados
encontrados = [m for m in marcadores if m in indices]
encontrados.sort(key=lambda k: indices[k])

if not encontrados:
    raise SystemExit("[ADVERTENCIA] No se encontró ningún marcador 3.1..3.18 en la columna A.")

# Si no se encontró 3.1, crear rango artificial 1..(fila anterior al primer marcador real)
if "3.1" not in indices:
    primera_fila_real = indices[encontrados[0]]
    if primera_fila_real > 1:
        indices["3.1"] = 1
        encontrados = ["3.1"] + encontrados
    # si el primer marcador real está en fila 1, no hay espacio para 3.1 artificial

# 3) Crear libro de salida (sin hoja INDEX)
wb_out = Workbook()
# Deja una hoja vacía por defecto que luego eliminamos cuando creemos la primera real
hoja_defecto = wb_out.active
hoja_defecto.title = "tmp"

# 4) Copiar bloques a hojas con el nombre del marcador
for idx, m in enumerate(encontrados):
    start_row = indices[m]
    if idx < len(encontrados) - 1:
        next_m = encontrados[idx + 1]
        end_row = indices[next_m] - 1
    else:
        end_row = max_r

    if end_row < start_row:
        print(f"[ADVERTENCIA] Rango vacío para {m} (start {start_row} > end {end_row}). Se omite.")
        continue

    # Crear hoja con nombre del marcador (evitando duplicados)
    sheet_name = m
    while sheet_name in wb_out.sheetnames:
        sheet_name += "_dup"
    ws_dest = wb_out.create_sheet(title=sheet_name)

    out_row = 1
    for r in range(start_row, end_row + 1):
        for c in range(1, max_c + 1):
            ws_dest.cell(row=out_row, column=c, value=ws_src.cell(row=r, column=c).value)
        out_row += 1

# Eliminar hoja temporal si ya creamos otras
if len(wb_out.sheetnames) > 1 and "tmp" in wb_out.sheetnames:
    std = wb_out["tmp"]
    wb_out.remove(std)

# 5) Guardar
wb_out.save(ruta_output)
print(f"[OK] Separación completada en: {ruta_output}")


[OK] Separación completada en: C:\Users\diego\OneDrive\9NO CICLO\ANALITICA DE DATOS\PARCIAL\PARTE 2\tarifarios_consolidado.xlsx


In [104]:
from openpyxl import load_workbook
import os
import unicodedata
import re

# === Ruta del archivo (mismo input y output; se sobreescribe) ===
ruta_xlsx = r"C:\Users\diego\OneDrive\9NO CICLO\ANALITICA DE DATOS\PARCIAL\PARTE 2\tarifarios_consolidado.xlsx"

# === Palabras clave EXACTAS (para secciones y para limpiar en Col C) ===
KEYS = [
    "TASAS",
    "INTERÉS MORATORIO",
    "COMISIONES",
    "GASTOS",
    "SEGUROS",
    "CONDICIONES DE PRIORITY PASS",
]

def normalize(s: str) -> str:
    """Quita tildes, pasa a mayúsculas y colapsa espacios para comparar de forma robusta."""
    if s is None:
        return ""
    s = str(s).strip()
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    s = re.sub(r"\s+", " ", s)
    return s.upper()

KEYS_NORM_SET = {normalize(k) for k in KEYS}

def last_data_row(ws) -> int:
    """Última fila que tiene al menos un valor en alguna columna."""
    max_r = ws.max_row or 0
    max_c = ws.max_column or 0
    for r in range(max_r, 0, -1):
        for c in range(1, max_c + 1):
            if ws.cell(row=r, column=c).value not in (None, ""):
                return r
    return 0

def collect_sections(ws):
    """
    Sobre la COLUMNA A ORIGINAL:
      - a1_value: valor original de A1
      - last_row: última fila con datos
      - sections: lista ordenada [(nombre_clave, fila_encabezado), ...]
        detectadas por igualdad al inicio (startswith) respecto a las KEYS.
    """
    a1_value = ws.cell(row=1, column=1).value
    lrow = last_data_row(ws)

    hits = []
    seen_norm = set()
    for r in range(1, lrow + 1):
        val = ws.cell(row=r, column=1).value
        val_norm = normalize(val)
        # Si la celda comienza con alguna KEY exacta (normalizada), la tomamos como encabezado
        for k in KEYS:
            k_norm = normalize(k)
            if k_norm in seen_norm:
                continue
            if val_norm.startswith(k_norm):
                hits.append((k, r))   # guarda el nombre "bonito" tal como en KEYS
                seen_norm.add(k_norm)
                break

    hits.sort(key=lambda x: x[1])
    return a1_value, lrow, hits

def main():
    if not os.path.exists(ruta_xlsx):
        raise SystemExit(f"[ERROR] No se encontró el archivo:\n{ruta_xlsx}")

    wb = load_workbook(ruta_xlsx, data_only=True)
    hojas = wb.sheetnames

    for name in hojas:
        ws = wb[name]

        # 1) Leer info basada en COLUMNA A ORIGINAL (antes de insertar columnas)
        a1_value, lrow, sections = collect_sections(ws)
        if lrow == 0:
            continue  # hoja vacía

        # 2) Insertar 2 columnas al inicio
        ws.insert_cols(1, amount=2)
        # Ahora:
        #   - Nueva Col A: repetirá A1 original
        #   - Nueva Col B: etiquetas por bloque
        #   - Col A original pasó a ser Col C (importante para limpieza)

        # 3) Rellenar nueva Col A con A1 original en todas las filas con datos
        for r in range(1, lrow + 1):
            ws.cell(row=r, column=1, value=a1_value)

        # 4) Etiquetar en nueva Col B por bloques, usando secciones detectadas en la A original
        #    Desde fila encabezado+1 hasta anterior al siguiente encabezado (último hasta lrow)
        if sections:
            for i, (label, start_header_row) in enumerate(sections):
                start = start_header_row + 1
                if i < len(sections) - 1:
                    end = sections[i + 1][1] - 1
                else:
                    end = lrow
                if end >= start:
                    for r in range(start, end + 1):
                        ws.cell(row=r, column=2, value=label)

        # 5) LIMPIEZA EN COLUMNA C:
        #    Borrar contenido SOLO si la celda es EXACTAMENTE una de las palabras clave.
        #    (Comparación normalizada por igualdad exacta; "tasas compensatorias" no se borra)
        lrow_after = last_data_row(ws)  # por si se expandió algo
        for r in range(1, lrow_after + 1):
            val_c = ws.cell(row=r, column=3).value  # Col C = Col A original
            if normalize(val_c) in KEYS_NORM_SET:
                ws.cell(row=r, column=3, value="")

        # 6) Eliminar la primera fila de la hoja (al final del proceso)
        ws.delete_rows(1, 1)

        print(f"[OK] Hoja '{name}': columnas agregadas, etiquetas aplicadas, Col C limpiada y primera fila eliminada.")

    # 7) Guardar (se sobreescribe el mismo archivo)
    wb.save(ruta_xlsx)
    print(f"[OK] Guardado en el MISMO archivo: {ruta_xlsx}")

if __name__ == "__main__":
    main()


[OK] Hoja '3.1': columnas agregadas, etiquetas aplicadas, Col C limpiada y primera fila eliminada.
[OK] Hoja '3.2': columnas agregadas, etiquetas aplicadas, Col C limpiada y primera fila eliminada.
[OK] Hoja '3.3': columnas agregadas, etiquetas aplicadas, Col C limpiada y primera fila eliminada.
[OK] Hoja '3.4': columnas agregadas, etiquetas aplicadas, Col C limpiada y primera fila eliminada.
[OK] Hoja '3.5': columnas agregadas, etiquetas aplicadas, Col C limpiada y primera fila eliminada.
[OK] Hoja '3.6': columnas agregadas, etiquetas aplicadas, Col C limpiada y primera fila eliminada.
[OK] Hoja '3.7': columnas agregadas, etiquetas aplicadas, Col C limpiada y primera fila eliminada.
[OK] Hoja '3.8': columnas agregadas, etiquetas aplicadas, Col C limpiada y primera fila eliminada.
[OK] Hoja '3.9': columnas agregadas, etiquetas aplicadas, Col C limpiada y primera fila eliminada.
[OK] Hoja '3.10': columnas agregadas, etiquetas aplicadas, Col C limpiada y primera fila eliminada.
[OK] Hoja

In [105]:
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import os

# === Archivo de entrada/salida (mismo archivo) ===
ruta_xlsx = r"C:\Users\diego\OneDrive\9NO CICLO\ANALITICA DE DATOS\PARCIAL\PARTE 2\tarifarios_consolidado.xlsx"

# Encabezados definitivos (A..H) en el orden exacto
HEADERS = [
    "TIPO_TARJETA",            # A
    "CATEGORÍA_INFO",          # B
    "TIPO_INFO",               # C
    "Porcentaje",              # D
    "MN",                      # E
    "Porcentaje",              # F
    "ME",                      # G
    "Observación y Vigencia",  # H
]

# Conjunto para detección exacta
HEADERS_SET = set(HEADERS)

def last_data_row(ws) -> int:
    """Última fila con al menos un valor."""
    max_r = ws.max_row or 0
    max_c = ws.max_column or 0
    for r in range(max_r, 0, -1):
        for c in range(1, max_c + 1):
            if ws.cell(row=r, column=c).value not in (None, ""):
                return r
    return 0

def main():
    if not os.path.exists(ruta_xlsx):
        raise SystemExit(f"[ERROR] No se encontró el archivo:\n{ruta_xlsx}")

    wb = load_workbook(ruta_xlsx, data_only=True)
    hojas = wb.sheetnames

    for name in hojas:
        ws = wb[name]

        max_r = last_data_row(ws)
        if max_r == 0:
            continue  # hoja vacía

        # 1) Eliminar la primera fila
        ws.delete_rows(1, 1)

        # 2) Insertar encabezados definitivos en nueva fila 1
        ws.insert_rows(1, 1)
        for col_idx, header in enumerate(HEADERS, start=1):
            ws.cell(row=1, column=col_idx, value=header)

        # Recalcular última fila con datos
        max_r = last_data_row(ws)
        if max_r < 2:
            print(f"[INFO] Hoja '{name}': solo encabezados tras limpieza.")
            continue

    wb.save(ruta_xlsx)
    print(f"[OK] Guardado en el MISMO archivo: {ruta_xlsx}")

if __name__ == "__main__":
    main()


[OK] Guardado en el MISMO archivo: C:\Users\diego\OneDrive\9NO CICLO\ANALITICA DE DATOS\PARCIAL\PARTE 2\tarifarios_consolidado.xlsx


In [106]:
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import os

ruta_xlsx = r"C:\Users\diego\OneDrive\9NO CICLO\ANALITICA DE DATOS\PARCIAL\PARTE 2\tarifarios_consolidado.xlsx"

def last_data_row(ws) -> int:
    max_r = ws.max_row or 0
    max_c = ws.max_column or 0
    for r in range(max_r, 0, -1):
        for c in range(1, max_c + 1):
            if ws.cell(row=r, column=c).value not in (None, ""):
                return r
    return 0

def mover_intrusos_segmentado(ws, col_idx, header_set, header_this):
    """
    En una columna: detecta TODAS las filas con "otro encabezado" y
    mueve solo el segmento desde esa fila hasta la fila anterior al siguiente intruso.
    """
    max_r = last_data_row(ws)
    # 1) ubicar todas las filas intrusas (arriba->abajo)
    intrusos = []
    for r in range(2, max_r + 1):
        v = ws.cell(row=r, column=col_idx).value
        if isinstance(v, str):
            s = v.strip()
            if s in header_set and (header_this is None or s != header_this):
                intrusos.append(r)
    if not intrusos:
        return

    # 2) procesar de ABAJO hacia ARRIBA para no alterar índices de segmentos superiores
    for i in range(len(intrusos) - 1, -1, -1):
        start = intrusos[i]
        end = (intrusos[i + 1] - 1) if i < len(intrusos) - 1 else max_r
        if end < start:
            continue
        col_letter = get_column_letter(col_idx)
        rango = f"{col_letter}{start}:{col_letter}{end}"
        ws.move_range(rango, rows=0, cols=1, translate=False)

def main():
    if not os.path.exists(ruta_xlsx):
        raise SystemExit(f"[ERROR] No se encontró el archivo:\n{ruta_xlsx}")

    wb = load_workbook(ruta_xlsx)
    for ws in wb.worksheets:
        max_r = last_data_row(ws)
        if max_r < 2:
            continue

        # Encabezados disponibles (fila 1, al menos A..H)
        headers = []
        for c in range(1, 9):
            v = ws.cell(row=1, column=c).value
            if isinstance(v, str) and v.strip():
                headers.append(v.strip())
        header_set = set(headers)

        # D..H (4..8): tratar cada columna por segmentos
        for col_idx in range(4, 9):
            header_this = ws.cell(row=1, column=col_idx).value
            header_this = header_this.strip() if isinstance(header_this, str) else None
            mover_intrusos_segmentado(ws, col_idx, header_set, header_this)

        print(f"[OK] Hoja '{ws.title}': intrusos segmentados movidos en D..H.")

    wb.save(ruta_xlsx)
    print(f"[OK] Guardado en el MISMO archivo: {ruta_xlsx}")

if __name__ == "__main__":
    main()


[OK] Hoja '3.1': intrusos segmentados movidos en D..H.
[OK] Hoja '3.2': intrusos segmentados movidos en D..H.
[OK] Hoja '3.3': intrusos segmentados movidos en D..H.
[OK] Hoja '3.4': intrusos segmentados movidos en D..H.
[OK] Hoja '3.5': intrusos segmentados movidos en D..H.
[OK] Hoja '3.6': intrusos segmentados movidos en D..H.
[OK] Hoja '3.7': intrusos segmentados movidos en D..H.
[OK] Hoja '3.8': intrusos segmentados movidos en D..H.
[OK] Hoja '3.9': intrusos segmentados movidos en D..H.
[OK] Hoja '3.10': intrusos segmentados movidos en D..H.
[OK] Hoja '3.11': intrusos segmentados movidos en D..H.
[OK] Hoja '3.12': intrusos segmentados movidos en D..H.
[OK] Hoja '3.13': intrusos segmentados movidos en D..H.
[OK] Hoja '3.14': intrusos segmentados movidos en D..H.
[OK] Hoja '3.15': intrusos segmentados movidos en D..H.
[OK] Hoja '3.16': intrusos segmentados movidos en D..H.
[OK] Hoja '3.17': intrusos segmentados movidos en D..H.
[OK] Hoja '3.18': intrusos segmentados movidos en D..H.
[

In [107]:
from openpyxl import load_workbook
import os

# === Archivo de entrada/salida (mismo archivo) ===
ruta_xlsx = r"C:\Users\diego\OneDrive\9NO CICLO\ANALITICA DE DATOS\PARCIAL\PARTE 2\tarifarios_consolidado.xlsx"

def last_data_row(ws) -> int:
    """Última fila con al menos una celda no vacía."""
    max_r = ws.max_row or 0
    max_c = ws.max_column or 0
    for r in range(max_r, 0, -1):
        for c in range(1, max_c + 1):
            if ws.cell(row=r, column=c).value not in (None, ""):
                return r
    return 0

def is_empty(value) -> bool:
    return value is None or (isinstance(value, str) and value.strip() == "")

def main():
    if not os.path.exists(ruta_xlsx):
        raise SystemExit(f"[ERROR] No se encontró el archivo:\n{ruta_xlsx}")

    wb = load_workbook(ruta_xlsx, data_only=True)
    hojas = wb.sheetnames

    for name in hojas:
        ws = wb[name]
        max_r = last_data_row(ws)
        if max_r < 2:
            continue  # hoja vacía o solo encabezado

        # 1) En columna D: eliminar filas con valor exactamente "Porcentaje"
        #    Recorremos de abajo hacia arriba para evitar saltos.
        deleted_count = 0
        for r in range(max_r, 2 - 1, -1):  # desde max_r hasta 2
            val = ws.cell(row=r, column=4).value  # Col D
            if isinstance(val, str) and val.strip() == "Porcentaje":
                ws.delete_rows(r, 1)
                deleted_count += 1
        if deleted_count:
            print(f"[{name}] Filas eliminadas por 'Porcentaje' en D: {deleted_count}")

        # Recalcular última fila por si cambió
        max_r = last_data_row(ws)
        if max_r < 2:
            continue

        # 2) Eliminar filas que contengan "col_" en cualquier columna
        deleted_col_ = 0
        for r in range(max_r, 1, -1):  # desde max_r hasta 2 (preservar encabezado)
            eliminar = False
            for c in range(1, ws.max_column + 1):
                v = ws.cell(row=r, column=c).value
                if isinstance(v, str) and "col_" in v:
                    eliminar = True
                    break
            if eliminar:
                ws.delete_rows(r, 1)
                deleted_col_ += 1
        if deleted_col_:
            print(f"[{name}] Filas eliminadas por contener 'col_': {deleted_col_}")

        # Recalcular última fila otra vez
        max_r = last_data_row(ws)
        if max_r < 2:
            continue

        # 3) Eliminar filas con D..H (4..8) TODAS vacías
        deleted_empty = 0
        for r in range(max_r, 1, -1):  # desde max_r hasta 2
            all_empty = True
            for c in range(4, 9):  # D..H
                v = ws.cell(row=r, column=c).value
                if not is_empty(v):
                    all_empty = False
                    break
            if all_empty:
                ws.delete_rows(r, 1)
                deleted_empty += 1
        if deleted_empty:
            print(f"[{name}] Filas eliminadas con D..H vacías: {deleted_empty}")

    wb.save(ruta_xlsx)
    print(f"[OK] Guardado en el MISMO archivo: {ruta_xlsx}")

if __name__ == "__main__":
    main()


[3.1] Filas eliminadas por 'Porcentaje' en D: 4
[3.1] Filas eliminadas por contener 'col_': 1
[3.1] Filas eliminadas con D..H vacías: 9
[3.2] Filas eliminadas por 'Porcentaje' en D: 4
[3.2] Filas eliminadas con D..H vacías: 9
[3.3] Filas eliminadas por 'Porcentaje' en D: 5
[3.3] Filas eliminadas por contener 'col_': 1
[3.3] Filas eliminadas con D..H vacías: 8
[3.4] Filas eliminadas por 'Porcentaje' en D: 3
[3.4] Filas eliminadas por contener 'col_': 2
[3.4] Filas eliminadas con D..H vacías: 11
[3.5] Filas eliminadas por 'Porcentaje' en D: 4
[3.5] Filas eliminadas por contener 'col_': 1
[3.5] Filas eliminadas con D..H vacías: 10
[3.6] Filas eliminadas por 'Porcentaje' en D: 4
[3.6] Filas eliminadas por contener 'col_': 1
[3.6] Filas eliminadas con D..H vacías: 11
[3.7] Filas eliminadas por 'Porcentaje' en D: 4
[3.7] Filas eliminadas por contener 'col_': 1
[3.7] Filas eliminadas con D..H vacías: 9
[3.8] Filas eliminadas por 'Porcentaje' en D: 3
[3.8] Filas eliminadas por contener 'col_':

In [108]:
from openpyxl import load_workbook
import os

# === Archivo de entrada/salida (mismo archivo) ===
ruta_xlsx = r"C:\Users\diego\OneDrive\9NO CICLO\ANALITICA DE DATOS\PARCIAL\PARTE 2\tarifarios_consolidado.xlsx"

def last_data_row(ws) -> int:
    """Última fila con al menos una celda no vacía."""
    max_r = ws.max_row or 0
    max_c = ws.max_column or 0
    for r in range(max_r, 0, -1):
        for c in range(1, max_c + 1):
            v = ws.cell(row=r, column=c).value
            if v is not None and (not isinstance(v, str) or v.strip() != ""):
                return r
    return 0

def is_empty(v) -> bool:
    return (v is None) or (isinstance(v, str) and v.strip() == "")

def main():
    if not os.path.exists(ruta_xlsx):
        raise SystemExit(f"[ERROR] No se encontró el archivo:\n{ruta_xlsx}")

    wb = load_workbook(ruta_xlsx, data_only=True)
    hojas = wb.sheetnames

    for name in hojas:
        ws = wb[name]
        max_r = last_data_row(ws)
        if max_r < 2:
            continue  # hoja vacía o solo encabezado

        # --- 1) Fill-down en columna C (TIPO_INFO) ---
        # Encabezado en fila 1, datos desde fila 2 en adelante
        for r in range(2, max_r + 1):
            val = ws.cell(row=r, column=3).value  # Col C
            if is_empty(val):
                prev = ws.cell(row=r - 1, column=3).value
                if not is_empty(prev):
                    ws.cell(row=r, column=3, value=prev)

        # --- 2) Eliminar filas donde alguna celda == encabezado de su misma columna ---
        max_r = last_data_row(ws)
        max_c = ws.max_column or 0
        headers = {}
        for c in range(1, max_c + 1):
            hv = ws.cell(row=1, column=c).value
            headers[c] = hv.strip() if isinstance(hv, str) else hv

        # Recolecta filas a eliminar (luego se borran bottom-up)
        rows_to_delete = set()
        for c in range(1, max_c + 1):
            header = headers.get(c, None)
            if header is None:
                continue
            for r in range(2, max_r + 1):
                v = ws.cell(row=r, column=c).value
                if isinstance(v, str):
                    if v.strip() == header:
                        rows_to_delete.add(r)

        # Borrar filas por encabezado encontrado (descendente)
        for r in sorted(rows_to_delete, reverse=True):
            ws.delete_rows(r, 1)

        # --- 3) Eliminar filas con F, G, H todas vacías ---
        max_r = last_data_row(ws)
        if max_r >= 2:
            for r in range(max_r, 1, -1):
                vF = ws.cell(row=r, column=6).value  # F
                vG = ws.cell(row=r, column=7).value  # G
                vH = ws.cell(row=r, column=8).value  # H
                if is_empty(vF) and is_empty(vG) and is_empty(vH):
                    ws.delete_rows(r, 1)

        print(f"[OK] Hoja '{name}': TIPO_INFO rellenado, filas con encabezados eliminadas, y filas con F-G-H vacías eliminadas.")

    wb.save(ruta_xlsx)
    print(f"[OK] Guardado en el MISMO archivo: {ruta_xlsx}")

if __name__ == "__main__":
    main()


[OK] Hoja '3.1': TIPO_INFO rellenado, filas con encabezados eliminadas, y filas con F-G-H vacías eliminadas.
[OK] Hoja '3.2': TIPO_INFO rellenado, filas con encabezados eliminadas, y filas con F-G-H vacías eliminadas.
[OK] Hoja '3.3': TIPO_INFO rellenado, filas con encabezados eliminadas, y filas con F-G-H vacías eliminadas.
[OK] Hoja '3.4': TIPO_INFO rellenado, filas con encabezados eliminadas, y filas con F-G-H vacías eliminadas.
[OK] Hoja '3.5': TIPO_INFO rellenado, filas con encabezados eliminadas, y filas con F-G-H vacías eliminadas.
[OK] Hoja '3.6': TIPO_INFO rellenado, filas con encabezados eliminadas, y filas con F-G-H vacías eliminadas.
[OK] Hoja '3.7': TIPO_INFO rellenado, filas con encabezados eliminadas, y filas con F-G-H vacías eliminadas.
[OK] Hoja '3.8': TIPO_INFO rellenado, filas con encabezados eliminadas, y filas con F-G-H vacías eliminadas.
[OK] Hoja '3.9': TIPO_INFO rellenado, filas con encabezados eliminadas, y filas con F-G-H vacías eliminadas.
[OK] Hoja '3.10': T

In [110]:
# filtrar_tarifarios_sobrescribir_con_validacion.py
import pandas as pd
from pathlib import Path
import numpy as np
import re

def filtrar_y_sobrescribir_consolidado(
    input_path: str,
    limite_caracteres: int = 500
):
    input_path = Path(input_path)
    xls = pd.ExcelFile(input_path)

    frames_limpios = []

    for hoja in xls.sheet_names:
        # Lee sin forzar dtype para no convertir NaN a "nan"
        df = pd.read_excel(xls, sheet_name=hoja)

        if df.empty:
            continue

        # Asegurar al menos 8 columnas (A..H) para evitar errores al validar
        if df.shape[1] < 8:
            # Agrega columnas vacías hasta llegar a 8
            faltan = 8 - df.shape[1]
            for i in range(faltan):
                df[f"_col_tmp_{i}"] = pd.NA

        # Columnas A-H por posición
        cols_ah = list(df.columns[:8])

        # Detectar celdas con >500 caracteres en A-H (ignora NaN y no-strings)
        def celda_larga(v):
            return isinstance(v, str) and len(v) > limite_caracteres

        mask_larga = df[cols_ah].applymap(celda_larga)
        filas_a_eliminar = mask_larga.any(axis=1)

        df_limpio = df.loc[~filas_a_eliminar].copy()

        # Guardar resultado de la hoja
        frames_limpios.append(df_limpio)

    # Consolidar todas las hojas limpias
    consolidado = pd.concat(frames_limpios, ignore_index=True) if frames_limpios else pd.DataFrame()

    if not consolidado.empty:
        # Asegurar al menos 8 columnas para la validación F/G/H
        if consolidado.shape[1] < 8:
            faltan = 8 - consolidado.shape[1]
            for i in range(faltan):
                consolidado[f"_col_tmpc_{i}"] = pd.NA

        # Indices de F (5), G (6), H (7) por posición
        cols = list(consolidado.columns)
        colF, colG, colH = cols[5], cols[6], cols[7]

        # Máscara: columna F contiene "Min" o "Max" como palabra (insensible a may/min)
        patron = re.compile(r"\b(min|max)\b", flags=re.IGNORECASE)
        mask_minmax = consolidado[colF].astype(str).str.contains(patron, na=False)

        if mask_minmax.any():
            # H = G
            consolidado.loc[mask_minmax, colH] = consolidado.loc[mask_minmax, colG]
            # G = F
            consolidado.loc[mask_minmax, colG] = consolidado.loc[mask_minmax, colF]
            # F vacía (opcional: np.nan o cadena vacía)
            consolidado.loc[mask_minmax, colF] = pd.NA

        # Si agregamos columnas temporales, puedes opcionalmente removerlas:
        # (solo si no pertenecen a tus encabezados reales)
        cols_finales = [c for c in consolidado.columns if not str(c).startswith(("_col_tmp_", "_col_tmpc_"))]
        consolidado = consolidado[cols_finales]

    # Sobrescribir el MISMO archivo con solo la hoja "Consolidado"
    with pd.ExcelWriter(input_path, engine="openpyxl", mode="w") as writer:
        consolidado.to_excel(writer, index=False, sheet_name="Consolidado")

    print(f"✅ Archivo sobrescrito con la hoja 'Consolidado': {input_path}")

if __name__ == "__main__":
    # === EDITA AQUÍ LA RUTA DEL ARCHIVO ===
    ruta_archivo = r"C:\Users\diego\OneDrive\9NO CICLO\ANALITICA DE DATOS\PARCIAL\PARTE 2\tarifarios_consolidado.xlsx"
    filtrar_y_sobrescribir_consolidado(ruta_archivo, limite_caracteres=500)


✅ Archivo sobrescrito con la hoja 'Consolidado': C:\Users\diego\OneDrive\9NO CICLO\ANALITICA DE DATOS\PARCIAL\PARTE 2\tarifarios_consolidado.xlsx


C:\Users\diego\AppData\Local\Temp\ipykernel_11504\1258021.py:37: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask_larga = df[cols_ah].applymap(celda_larga)
C:\Users\diego\AppData\Local\Temp\ipykernel_11504\1258021.py:37: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask_larga = df[cols_ah].applymap(celda_larga)
C:\Users\diego\AppData\Local\Temp\ipykernel_11504\1258021.py:37: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask_larga = df[cols_ah].applymap(celda_larga)
C:\Users\diego\AppData\Local\Temp\ipykernel_11504\1258021.py:37: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask_larga = df[cols_ah].applymap(celda_larga)
C:\Users\diego\AppData\Local\Temp\ipykernel_11504\1258021.py:37: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask_larga = df[cols_ah].applymap(celda_larga)
C:\Users\diego\AppDa